In [1]:
import os
import sys

# 1️⃣ Chuyển working directory tới root project (nơi có thư mục src)
# %cd /content/drive/MyDrive/chatbot
%cd /workspace/chatbot
# 2️⃣ Thêm folder root vào sys.path để Python tìm src
# sys.path.append(os.getcwd())

# 3️⃣ Kiểm tra
!ls
# Nên thấy thư mục src, app_gradio.py, notebooks, v.v.


/workspace/chatbot
LICENSE			 data_generating	 rag_evaluation.ipynb
README.md		 data_viz.ipynb		 src
app_gradio.ipynb	 eval_result		 train_models.ipynb
app_gradio_server.ipynb  model_evaluation.ipynb
data			 models


In [2]:
import sys
import subprocess

def pip_install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + package.split())

# ============================
# 1) Install core libraries
# ============================
pip_install("langchain langchain-community langchain-google-genai google-generativeai transformers torch sentence-transformers pinecone gradio peft bitsandbytes accelerate pymongo")

# ============================
# 2) Upgrade pinecone
# ============================
pip_install("pinecone --upgrade")

# ============================
# 3) Upgrade bitsandbytes
# ============================
pip_install("-U bitsandbytes")

# ============================
# 4) Rank BM25 + unidecode
# ============================
pip_install("rank-bm25 unidecode")

# ============================
# 5) pymongo (redundant but safe)
# ============================
pip_install("pymongo")

# ============================
# 6) Install đúng Gradio 5.50.0
# ============================
pip_install("gradio==5.50.0")


  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You mig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [rank-bm25]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 31.1 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 23.5 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: pydantic-core━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [ruff]
    Found existing installation: pydantic_core 2.41.5━━━━━━━━━ 1/6 [ruff]
    Uninstalling pydantic_core-2.41.5:━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [ruff]
      Successfully uninstalled pydantic_core-2.41.5━━━━━━━━━━━ 1/6 [ruff]
  Attempting uninstall: pydantic━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [ruff]
    Found existing installation: pydantic 2.12.4━━━━━━━━━━━━━━ 1/6 [ruff]
    Uninstalling pydantic-2.12.4:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [ruff]
      Successfully uninstalled pydantic-2.12.4━━━━━━━━━━━━━━━━ 1/6 [ruff]
  Attempting uninstall: gradio-client╺━━━━━━━━━━━━━━━━━━━ 3/6 [pydantic]
    Found existing installation: gradio_client 2.0.1━━━━━━━━━━ 3/6 [pydantic]
  

In [ ]:
import os
import gradio as gr

from src.logic_module import (
    list_sessions,
    create_session_handler,
    load_session_handler,
    delete_session_handler,
    chat_fn
)

# -------------------------- 1️⃣ Gradio UI --------------------------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 💬 Vietnam Legal Assistance")

    with gr.Row():
        # Sidebar: session management
        with gr.Column(scale=1):
            session_dropdown = gr.Dropdown(
                label="Select Session",
                choices=[lbl for lbl, _ in list_sessions()],
                interactive=True
            )
            new_name = gr.Textbox(label="New Session Name (optional)", placeholder="Enter new session name...")
            create_btn = gr.Button("➕ Create New")
            load_btn = gr.Button("📂 Load Session")
            delete_btn = gr.Button("🗑️ Delete Session")
            refresh_btn = gr.Button("🔄 Refresh List")

        # Main chat area
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=480)
            refs_output = gr.HTML()
            with gr.Row():
                msg = gr.Textbox(label="Question", placeholder="Type your question here...", lines=2)
                send = gr.Button("📨 Send")
            clear_btn = gr.Button("🧹 Clear Chat (local)")

    # -------------------------- 2️⃣ State --------------------------
    state_session = gr.State(value=None)
    state_cache = gr.State(value=[])  # retrieval cache

    # -------------------------- 3️⃣ Events --------------------------

    # Refresh session list
    refresh_btn.click(
        lambda: gr.update(choices=[lbl for lbl,_ in list_sessions()]),
        None,
        session_dropdown
    )

    # Create new session
    create_btn.click(
        create_session_handler,
        inputs=[new_name],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Load existing session
    load_btn.click(
        load_session_handler,
        inputs=[session_dropdown],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Delete session
    delete_btn.click(
        delete_session_handler,
        inputs=[session_dropdown],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Send chat message
    send.click(
        chat_fn,
        inputs=[state_session, chatbot, msg, state_cache],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )
    msg.submit(
        chat_fn,
        inputs=[state_session, chatbot, msg, state_cache],
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Clear chat
    clear_btn.click(
        lambda: (None, [], [], ""),
        outputs=[state_session, chatbot, state_cache, refs_output]
    )

    # Auto-load session list when UI starts
    def load_session_list():
        return gr.update(choices=[lbl for lbl, _ in list_sessions()])

    demo.load(
        load_session_list,
        inputs=None,
        outputs=session_dropdown
    )

# -------------------------- 4️⃣ Launch --------------------------
if __name__ == "__main__":
    demo.launch(server_name="127.0.0.1", server_port=7861, debug=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

/venv/main/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipykernel_1176/1546606495.py:13: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipykernel_1176/1546606495.py:32: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=480)
/tmp/ipykerne

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d2fc6b03a9a1797de5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


# Del Session

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

uri = os.getenv("MONGO_URI")
db_name = os.getenv("MONGO_DB")
col_name = os.getenv("MONGO_COLLECTION")

from pymongo import MongoClient
result = MongoClient(uri)[db_name][col_name].delete_many({})
print("Đã xoá", result.deleted_count, "document.")


Đã xoá 35 document.
